# Boston Housing

In [10]:
include("../src/load.jl")
using AlgorithmicRecourse, MLDatasets, Flux
using Plots, PlotThemes
theme(:juno)
using Logging
disable_logging(Logging.Info)
output_folder = "output/boston_housing_ensemble"

"output/boston_housing_ensemble"

## Training the classifier

In [11]:
using MLDatasets, Statistics
X = BostonHousing.features()
y = BostonHousing.targets()
y = y .>= median(y); # binary target

To start off we will just train a single neural network for the binary classification task.

In [12]:
# Prepare data and model:
using Random
Random.seed!(1234)
using StatsBase
dt = fit(ZScoreTransform, X, dims=2)
StatsBase.transform!(dt, X)
xs = Flux.unstack(X,2)
data = zip(xs,y)
nn = Models.build_model(input_dim=size(X)[1], n_hidden=100)
loss(x, y) = Flux.Losses.logitbinarycrossentropy(nn(x), y)

loss (generic function with 1 method)

The model achieves decent training accuracy

In [13]:
run = false
if run
  # Train model:
  using Flux.Optimise: update!, ADAM
  using Statistics, StatsBase
  opt = ADAM()
  epochs = 100
  avg_loss(data) = mean(map(d -> loss(d[1],d[2]), data))
  accuracy(data) = sum(map(d ->round.(Flux.σ.(nn(d[1]))) .== d[2], data))[1]/length(data)

  using Plots
  anim = Animation()
  avg_l = [avg_loss(data)]
  p1 = scatter( ylim=(0,avg_l[1]), xlim=(0,epochs), legend=false, xlab="Epoch", title="Average loss")
  acc = [accuracy(data)]
  p2 = scatter( ylim=(0.5,1), xlim=(0,epochs), legend=false, xlab="Epoch", title="Accuracy")

  for epoch = 1:epochs
    for d in data
      gs = gradient(params(nn)) do
        l = loss(d...)
      end
      update!(opt, params(nn), gs)
    end
    avg_l = vcat(avg_l,avg_loss(data))
    plot!(p1, [0:epoch], avg_l, color=1)
    scatter!(p1, [0:epoch], avg_l, color=1)
    acc = vcat(acc,accuracy(data))
    plot!(p2, [0:epoch], acc, color=1)
    scatter!(p2, [0:epoch], acc, color=1)
    plt=plot(p1,p2, size=(600,300))
    frame(anim, plt)
  end

  gif(anim, "www/boston_housing_single_nn.gif", fps=10);
end

![](www/boston_housing_single_nn.gif)

Next we will build and train a deep ensemble.

In [14]:
run = false
if run
    K = 50
    𝓜 = Models.build_ensemble(K,kw=(input_dim=size(X)[1], n_hidden=100));
    𝓜, anim = Models.forward(𝓜, data, opt, n_epochs=30, plot_every=10); # fit the ensemble
    Models.save_ensemble(𝓜, root=output_folder#############
    
    
    ) # save to disk
    gif(anim, "www/boston_housing_ensemble_loss.gif", fps=25);
end

![](www/boston_housing_ensemble_loss.gif)

In [15]:
# 𝓜 = Models.load_ensemble(root=output_folder)
# 𝑴=Models.FittedEnsemble(𝓜);
# using Random
# Random.seed!(1234)
# x̅ = X[:,(y.==0)'][:,rand(1:length((y.==0)'))] # select individual sample
# x̅ = reshape(x̅, (length(x̅),1))
# γ = 0.75
# target=1.0


## Experiment

Prepare ensemble for use with AlgorithmicRecourse.jl:

In [16]:
𝓜 = Models.load_ensemble(root=output_folder)
𝑴 = Models.FittedEnsemble(𝓜, Flux.ADAM, :logitbinarycrossentropy);

Prepare grid of variables for experiment:

In [20]:
using .Experiments: GridVariables, Experiment
# Variables:
μ = [0.01,0.05,0.1]
γ = [0.55,0.75,0.9]
grid = GridVariables(μ, γ)
experiment = Experiment(X,y,𝑴,target,grid,n_rounds)

GridVariables([0.01, 0.05, 0.1], [0.55, 0.75, 0.9])

In [ ]:
# run_experiment(experiment, generator, 5)